In [ ]:
#!python -m pip install ipython

# Data for the model

In [35]:
#Import Libraries

import PIL
from PIL import Image
import numpy as np
from numpy import clip
import glob

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

from patchify import patchify, unpatchify
import matplotlib.pyplot as plt
import os
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [36]:
def pre_processing(input_images,output1_images,output2_images):
    input_list = []
    output1_list = []
    output2_list = []
    for i,j,k in zip(input_images,output1_images,output2_images):
        image = Image.open(i)
        arr_image = np.asarray(image) #image to array
        input_list.append(arr_image);

        image = Image.open(j)
        arr_image = np.asarray(image) #image to array
        output1_list.append(arr_image);

        image = Image.open(k)
        arr_image = np.asarray(image) #image to array
        output2_list.append(arr_image);
    return input_list, output1_list,output2_list

In [37]:
#Function that uses patchify to slice the images into given window size
#parameters 1.list of images(as an array of pixels) 2.common prefix for the patched images 3. name of the folder to place the patches
def image_split(large_image_stack,window_size,step_size):
    all_img_patches = []
    for k in range(len(large_image_stack)):
        large_image = large_image_stack[k]
        patches_img = patchify(large_image, (window_size, window_size), step=step_size)  # no overlap
        for i in range(patches_img.shape[0]):
            for j in range(patches_img.shape[1]):
                single_patch_img = patches_img[i,j,:,:]
                all_img_patches.append(single_patch_img)
    images = np.array(all_img_patches)
    return images

In [38]:
# Running Just Patchify+ImageAugment to augment the image of size 256 x 256 and step = 51
#folder name = /home/lakram9u/py_scripts/D2DB/Generated_Data/Patchify_ImgGen_256x256


input_images = glob.glob("INSERT YOUR PATH")
output1_images = glob.glob("INSERT YOUR PATH")
output2_images = glob.glob("INSERT YOUR PATH")

#converting the images to arrays
input_list, output1_list,output2_list = pre_processing(input_images,output1_images,output2_images)

print(len(input_list),len(output1_list),len(output2_list))
print("Performing Image Splitting......\n")

#Patchify
window_size = 256
step_size = 14

input_split_images = image_split(input_list,
                                window_size,
                                step_size)

output1_split_images = image_split(output1_list,
                                  window_size,
                                  step_size)

output2_split_images = image_split(output2_list,                                   
                                   window_size,
                                   step_size)
print("Done:)")

7 7 7
Performing Image Splitting......

Done:)


In [39]:
#Adding 3rd dimension for channel
input_list = []
output1_list = []
output2_list = []
for i in range(input_split_images.shape[0]):
    arr_image = input_split_images[i]/255.0
    input_array = arr_image.reshape(arr_image.shape + (1,))
    input_list.append(input_array);
    
    arr_image = output1_split_images[i]/255.0
    input_array = arr_image.reshape(arr_image.shape + (1,))
    output1_list.append(input_array);
    
    arr_image = output2_split_images[i]/255.0
    input_array = arr_image.reshape(arr_image.shape + (1,))
    output2_list.append(input_array);

In [40]:
input_list = np.array(input_list)
output1_list = np.array(output1_list)
output2_list = np.array(output2_list)

In [41]:
output1_list.shape

(2527, 256, 256, 1)

In [42]:
#Train test split
from sklearn.model_selection import train_test_split
inp, inp_test, out1, out1_test, out2, out2_test = train_test_split(input_list, output1_list, output2_list, test_size=0.2, random_state=42)
inp_train,inp_val,out1_train,out1_val,out2_train,out2_val = train_test_split(inp, out1, out2, test_size=0.2, random_state=42)

In [43]:
print(inp_train.shape,inp_val.shape,inp_test.shape)

(1616, 256, 256, 1) (405, 256, 256, 1) (506, 256, 256, 1)


# Model Training

In [10]:
import tensorflow as tf

import os
import pathlib
import time
import datetime

from matplotlib import pyplot as plt
from IPython import display

In [11]:
#Training set size
BUFFER_SIZE = 1616
# The batch size of 1 produced better results for the U-Net in the original pix2pix experiment
BATCH_SIZE = 1
# Each image is 256x256 in size
IMG_WIDTH = 256
IMG_HEIGHT = 256
OUTPUT_CHANNELS = 1
LAMBDA = 100


In [12]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [13]:
def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [14]:
def Generator():
  inputs = tf.keras.layers.Input(shape=[256, 256, 1])

#Encoder
  down_stack = [
    downsample(64, 4, apply_batchnorm=False),  # (batch_size, 128, 128, 64)
    downsample(128, 4),  # (batch_size, 64, 64, 128)
    downsample(256, 4),  # (batch_size, 32, 32, 256)
    downsample(512, 4),  # (batch_size, 16, 16, 512)
    downsample(512, 4),  # (batch_size, 8, 8, 512)
    downsample(512, 4),  # (batch_size, 4, 4, 512)
    downsample(512, 4),  # (batch_size, 2, 2, 512)
    downsample(512, 4),  # (batch_size, 1, 1, 512)
  ]
#Decoder
  up_stack = [
    upsample(512, 4, apply_dropout=True),  # (batch_size, 2, 2, 1024)
    upsample(512, 4, apply_dropout=True),  # (batch_size, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True),  # (batch_size, 8, 8, 1024)
    upsample(512, 4),  # (batch_size, 16, 16, 1024)
    upsample(256, 4),  # (batch_size, 32, 32, 512)
    upsample(128, 4),  # (batch_size, 64, 64, 256)
    upsample(64, 4),  # (batch_size, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh')  # (batch_size, 256, 256, 3)

  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [15]:
generator = Generator()
#tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

In [16]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def generator_loss(disc_generated_output, gen_output, target):
    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

    # Mean absolute error
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

    total_gen_loss = gan_loss + (LAMBDA * l1_loss)

    return total_gen_loss, gan_loss, l1_loss

In [17]:
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[256, 256, 1], name='input_image')
  tar = tf.keras.layers.Input(shape=[256, 256, 1], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar])  # (batch_size, 256, 256, channels*2)

  down1 = downsample(64, 4, False)(x)  # (batch_size, 128, 128, 64)
  down2 = downsample(128, 4)(down1)  # (batch_size, 64, 64, 128)
  down3 = downsample(256, 4)(down2)  # (batch_size, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (batch_size, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1)  # (batch_size, 31, 31, 512)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (batch_size, 33, 33, 512)

  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2)  # (batch_size, 30, 30, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [18]:
discriminator = Discriminator()
#tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

In [19]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

In [20]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [21]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [22]:
#import numpy as np 
#inp = np.random.rand(1,256,256,1)
#prediction = generator(inp)


In [23]:
log_dir="logs/"

summary_writer = tf.summary.create_file_writer(log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [24]:
def train_step(input_image, target, step):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)
    
    gen_output = tf.cast(gen_output, tf.float64)
    disc_generated_output = tf.cast(disc_generated_output,tf.float64)
    disc_real_output = tf.cast(disc_real_output,tf.float64)
    
    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output[0], gen_output[0], target[0])
    disc_loss = discriminator_loss(disc_real_output[0], disc_generated_output[0])

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

  with summary_writer.as_default():
    tf.summary.scalar('gen_total_loss', gen_total_loss, step=step//1616)
    tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=step//1616)
    tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=step//1616)
    tf.summary.scalar('disc_loss', disc_loss, step=step//1616)

In [25]:
def fit(inp_ds, out_ds, steps):
  
  start = time.time()
  for step, (input_image, target) in zip(range(steps), zip(inp_ds, out_ds)):
    if (step) % 1616 == 0:

      if step != 0:
        print(f'Time taken for epoch {step/1616:.2f}: {time.time()-start:.2f} sec\n')
    input_image = tf.reshape(input_image, (1, 256, 256, 1))
    target = tf.reshape(target,(1,256,256,1))
    train_step(input_image, target, step)

    # Training step
    if (step+1) % 10 == 0:
      print('.', end='', flush=True)


    # Save (checkpoint) the model every epoch
    if (step) % 1616 == 0:
      checkpoint.save(file_prefix=checkpoint_prefix)
  end = time.time()
  time_taken = end - start
  print(f"Time taken: {time_taken:.6f} seconds") 

In [27]:
!kill 3107615

/bin/sh: line 0: kill: (3107615) - No such process


In [28]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 3107615), started 0:09:11 ago. (Use '!kill 3107615' to kill it.)

In [31]:
inp_train = np.repeat(inp_train, 50, axis=0)
out1_train = np.repeat(out1_train,50,axis=0)
#fit(inp_train, out1_train, steps=80800)
print(inp_train.shape,out1_train.shape)

(80800, 256, 256, 1) (80800, 256, 256, 1)


In [32]:
fit(inp_train, out1_train, steps=80800)

.................................................................................................................................................................Time taken for epoch 1.00: 10978.19 sec

..................................................................................................................................................................Time taken for epoch 2.00: 17665.20 sec

.................................................................................................................................................................Time taken for epoch 3.00: 26568.93 sec

..................................................................................................................................................................Time taken for epoch 4.00: 32828.56 sec

..................................................................................................................................................................Time taken for epoch 5.00: 4

In [44]:
out1_pred = generator(inp_test,training=True)

In [49]:
out1_pred = out1_pred.numpy()

In [50]:
out1_pred.shape

(506, 256, 256, 1)